In [ ]:
from tqdm import tqdm
from scipy.stats import chi2
from num2words import num2words
from collections import Counter
from scipy import spatial, sparse
from nltk.corpus import stopwords
from itertools import combinations
from nltk.stem import PorterStemmer
from sklearn.externals import joblib 
from nltk.tokenize import word_tokenize
# from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import KFold
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, cosine_distances

import os
import spacy
import imp
import gzip
import copy
import nltk
import pickle
import scipy
import string
import gensim
import operator
import datetime
import multiprocessing
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
#import LDA_ELJST as lda
import ELJST_script_unigram as lda
#import LJST_script_BTM as lda
#import ELJST_script_BTM as lda
import matplotlib.pyplot as plt

In [ ]:
import utils as my_utils

In [ ]:
from scipy import spatial, sparse
from scipy.stats import chi2
from num2words import num2words
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bs4 import BeautifulSoup as bs

import copy
import nltk
import scipy
import multiprocessing

import numpy as np
import pandas as pd

stop_words = stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()
stemmer = nltk.stem.PorterStemmer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()


In [ ]:
def get_edges(i):
    t = np.where(i>0)[0]
    comb = combinations(t, 2)    
    embeds = {j:[] for j in t}

    for p, q in comb:
        if word_similarity[p][q]:
            embeds[p] += [q]
            embeds[q] += [p]
    return embeds

In [ ]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in tqdm(parse(path)):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
def process_df(df):
    df['text'] = preprocess(df['reviewText'])
    
#     pool = multiprocessing.Pool(n_cores)
#     df['cleaned'] = pool.map(process_l, df['text'].tolist())
#     pool.close()
    
#     df['text'] = df['cleaned'].apply(lambda x: " ".join(x))
    return df

In [ ]:
# dataset_ = getDF('datasets/reviews_Electronics_5.json.gz')
# dataset_.shape

In [ ]:
# dataset_.head(10000).to_pickle("resources/electronics_10k")

In [ ]:
# dataset = dataset_.head(100)

In [ ]:
dataset = pd.read_pickle("resources/electronics_10k")

In [ ]:
def preprocess(pd):
    pd = pd.str.lower()
    pd = pd.str.replace('[{}]'.format('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\n\t'), ' ')
    pd = pd.str.replace('\d+', ' ')
    pd = pd.apply(lambda x: [w for w in w_tokenizer.tokenize(x)])
#     pd = pd.apply(lambda x: convert_numbers(x))
    pd = pd.str.join(' ')
    
    pd = pd.apply(lambda x: [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(x)])    
    pd = pd.apply(lambda x: [lemmatizer.lemmatize(w, 'v') for w in x])
    pd = pd.apply(lambda x: [item for item in x if item not in stop_words])
    pd = pd.apply(lambda x: [item for item in x if len(item)>2])
    pd = pd.apply(lambda x: " ".join(x))
    return pd

In [ ]:
%%time
dataset = process_df(dataset)

In [ ]:
n_cores = 30

In [ ]:
n = int(dataset.shape[0]/n_cores)
list_df = [dataset[i:i+n] for i in range(0, dataset.shape[0],n)]

pool = multiprocessing.Pool(n_cores)
processed_list_df = pool.map(process_df, list_df)
pool.close()

dataset = pd.concat(processed_list_df)
dataset.shape

In [ ]:
Counter(dataset.overall)

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r', encoding='utf8')
    model = {}
    for line in tqdm(f):
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

In [ ]:
# %%time
# embedding_dim = 300
# embeddings_index = loadGloveModel("nongit_resources/glove.6B.300d.txt")

In [ ]:
n_topics = 5
n_sentiment = 5
alpha = 0.1/n_topics * np.ones(n_topics)
beta = .01
gamma = 10
gamma = [gamma/(n_topics*n_sentiment)]*n_sentiment

maxiter = 20
lambda_param = 1.0

In [ ]:
gamma

In [ ]:
imp.reload(lda)

In [ ]:
sampler = lda.SentimentLDAGibbsSampler(n_topics, alpha, beta, gamma, numSentiments=n_sentiment, minlabel = 0, 
                                       maxlabel = 5, SentimentRange = 5, max_df = .5, min_df = 5, 
                                       lambda_param = lambda_param)

In [ ]:
%%time
sampler._initialize_(reviews = dataset.text.tolist(), labels = dataset.overall.tolist())

In [ ]:
# sampler.wordOccuranceMatrix.sum()

In [ ]:
# sampler.words

In [ ]:
# %%time
# word_embeddings = []

# for word in tqdm(sampler.words):
#     emb = embeddings_index.get(word, np.array([0]*embedding_dim))
#     word_embeddings.append(emb.tolist())

# word_embeddings = np.array(word_embeddings)

In [ ]:
# cutoff = .35
# word_similarity = cosine_similarity(word_embeddings)
# word_similarity = word_similarity > cutoff
# word_similarity = word_similarity.astype(int)
# np.fill_diagonal(word_similarity, 0)

In [ ]:
# wordOccuranceMatrixBinary = sampler.wordOccuranceMatrix.copy()
# wordOccuranceMatrixBinary[wordOccuranceMatrixBinary > 1] = 1
# wordOccuranceMatrixBinary.shape

In [ ]:
# wordOccuranceMatrixBinary[0].sum()

In [ ]:
# np.sum(wordOccuranceMatrixBinary)

In [ ]:
# Counter(np.array([i[1] for i in nltk.pos_tag(sampler.words)]))

In [ ]:
# pp = np.array([i[1] for i in nltk.pos_tag(sampler.words)])
# pp[pp=='JJ'] = 1
# pp[pp=='JJR'] = 1
# pp[pp=='JJS'] = 1
# pp[pp=='NN'] = 1
# pp[pp=='NNS'] = 1
# pp[pp=='NNP'] = 1
# pp[pp=='NNPS'] = 1
# pp[pp!='1'] = 0
# pp = pp.astype(int)

In [ ]:
# pp.sum()

In [ ]:
# np.where(pp!=1)[0].shape

In [ ]:
# wordOccuranceMatrixBinary[:, np.where(pp!=1)[0]] = 0
# np.sum(wordOccuranceMatrixBinary)

In [ ]:
# %%time
# pool = multiprocessing.Pool(n_cores)
# similar_words = pool.map(get_edges, wordOccuranceMatrixBinary)
# pool.close()
# pickle_out = open("resources/amazon_electronics_glove_0.35_POS_10k.pickle","wb")
# pickle.dump(similar_words, pickle_out)
# pickle_out.close()

In [ ]:
similar_words = pickle.load(open("resources/amazon_electronics_glove_0.35_POS_10k.pickle","rb"))

In [ ]:
sampler.run(reviews=dataset.text.tolist(), labels=dataset.overall.tolist(), 
            similar_words=similar_words, mrf=True, maxIters=20)

In [ ]:
Counter([len(i) for i in sampler.docs_edges])

In [ ]:
plt.plot(sampler.loglikelihood_history)

In [ ]:
silhouette_score(euclidean_distances(sampler.wordOccuranceMatrix),
                 sampler.dt_distribution.argmax(axis=1), metric='precomputed')

In [ ]:
davies_bouldin_score(sampler.wordOccuranceMatrix, sampler.dt_distribution.argmax(axis=1))

In [ ]:
my_utils.coherence_score(sampler.wordOccuranceMatrix, list(sampler.getTopKWords(5).values()), sampler.vocabulary)

In [ ]:
%%time
my_utils.get_hscore_multi(sampler.dt_distribution, sampler.wordOccuranceMatrix, n_topics, 3000)

In [ ]:
sampler.loglikelihood()

In [ ]:
sampler.perplexity()

### Appendix

In [ ]:
# p = [item for sublist in dataset['cleaned'].tolist() for item in sublist]

In [ ]:
# sorted(Counter(p))

In [ ]:
# def process_l(s):
#     return [i.lemma_ for i in sp(s) if i.lemma_ not in '-PRON-']

In [ ]:
# l = dataset['text'].tolist()

In [ ]:
# pool = multiprocessing.Pool(n_cores)
# processed_l = pool.map(process_l, l)
# pool.close()

In [ ]:
# joblib.dump(sampler, "resources/sampler_20iter_0.5_1")

In [ ]:
# pickle_out = open("resources/amazon_muiscal_glove_0.4.pickle","wb")
# pickle.dump(similar_words, pickle_out)
# pickle_out.close()